In [18]:
from midiutil import MIDIFile
from mingus.core import chords


NOTES = ['C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'Ab', 'A', 'Bb', 'B']
OCTAVES = list(range(11))
NOTES_IN_OCTAVE = len(NOTES)

errors = {
    'notes': 'Bad input, please refer this spec-\n'
}


def swap_accidentals(note):
    if note == 'Db':
        return 'C#'
    if note == 'D#':
        return 'Eb'
    if note == 'E#':
        return 'F'
    if note == 'Gb':
        return 'F#'
    if note == 'G#':
        return 'Ab'
    if note == 'A#':
        return 'Bb'
    if note == 'B#':
        return 'C'

    return note


def note_to_number(note: str, octave: int) -> int:
    note = swap_accidentals(note)
    assert note in NOTES, errors['notes']
    assert octave in OCTAVES, errors['notes']

    note = NOTES.index(note)
    note += (NOTES_IN_OCTAVE * octave)

    assert 0 <= note <= 127, errors['notes']

    return note


chord_progression = ["Cmaj7", "Cmaj7", "Fmaj7", "Gdom7"]

array_of_notes = []
for chord in chord_progression:
    array_of_notes.extend(chords.from_shorthand(chord))

array_of_note_numbers = []
for note in array_of_notes:
    OCTAVE = 4
    array_of_note_numbers.append(note_to_number(note, OCTAVE))

track = 0
channel = 0
time = 0  # In beats
duration = 1  # In beats
tempo = 120  # In BPM
volume = 100  # 0-127, as per the MIDI standard

MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
# automatically)
MyMIDI.addTempo(track, time, tempo)

for i, pitch in enumerate(array_of_note_numbers):
    MyMIDI.addNote(track, channel, pitch, time + i, duration, volume)

with open("test.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

In [19]:
# Load dataframe
import pandas as pd

data_file = 'pmn_timeadjusted/max_results_20240409.csv'
df = pd.read_csv(data_file,)

# Change name of the first column to time
df.columns = ['time'] + list(df.columns[1:])
freq_bins = df.columns[1:]
df

,time,0-1500,1500-5000,5000-10000,10k-20000,20k-60000
0,0.00,18248.373448,2901.485958,19907.483676,127319.500410,166469.213110
1,0.05,18882.981538,2372.933408,24599.109265,153929.658636,210072.506514
2,0.10,24805.913706,3850.990234,21728.549326,123848.284920,135055.410925
3,0.15,32262.614614,3339.338226,31683.330300,159345.526698,198820.016215
4,0.20,41720.110299,1893.650360,23695.078882,149188.548702,194030.773951
...,...,...,...,...,...,...
283,23.35,133217.246959,112737.039800,32460.885224,123293.269207,78906.718787
284,23.40,81211.232929,63862.883906,27624.045446,56779.877672,75969.104318
285,23.45,65772.340591,40722.700219,28707.998790,90532.229732,126940.093780
286,23.50,66818.392993,60006.556283,19007.769207,104873.355677,157320.342333


In [51]:
for col in freq_bins:
    # Scale the column to 0 - 64
    df[col+"_scaled"] = (df[col] - df[col].min()) / (df[col].max() - df[col].min()) * 16

    # Round the scaled column to the nearest power of 2
    df[col+"_scaled"] = 2**df[col+"_scaled"].pow(1/2).round()



df


,time,0-1500,1500-5000,5000-10000,10k-20000,20k-60000,0-1500_scaled,1500-5000_scaled,5000-10000_scaled,10k-20000_scaled,20k-60000_scaled
0,0.00,18248.373448,2901.485958,19907.483676,127319.500410,166469.213110,2.0,1.0,2.0,8.0,16.0
1,0.05,18882.981538,2372.933408,24599.109265,153929.658636,210072.506514,2.0,1.0,2.0,8.0,16.0
2,0.10,24805.913706,3850.990234,21728.549326,123848.284920,135055.410925,2.0,1.0,2.0,8.0,8.0
3,0.15,32262.614614,3339.338226,31683.330300,159345.526698,198820.016215,2.0,1.0,2.0,8.0,16.0
4,0.20,41720.110299,1893.650360,23695.078882,149188.548702,194030.773951,2.0,1.0,2.0,8.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...
283,23.35,133217.246959,112737.039800,32460.885224,123293.269207,78906.718787,4.0,8.0,2.0,8.0,4.0
284,23.40,81211.232929,63862.883906,27624.045446,56779.877672,75969.104318,4.0,4.0,2.0,4.0,4.0
285,23.45,65772.340591,40722.700219,28707.998790,90532.229732,126940.093780,4.0,4.0,2.0,4.0,8.0
286,23.50,66818.392993,60006.556283,19007.769207,104873.355677,157320.342333,4.0,4.0,2.0,8.0,8.0


In [52]:

chord_progression = ["Cmaj7", "Dmin7", "Emin7", "Fmaj7"]

array_of_notes = []

for chord in chord_progression:
    array_of_notes.extend(chords.from_shorthand(chord) * 8)

array_of_note_numbers = []
for note in array_of_notes:
    OCTAVE = 4
    array_of_note_numbers.append(note_to_number(note, OCTAVE))

track = 0
channel = 0
time = 0  # In beats
duration = 1  # In beats
tempo = 150 # In BPM
volume = 100  # 0-127, as per the MIDI standard

MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
# automatically)
MyMIDI.addTempo(track, time, tempo)



In [54]:
cur = 0

for i,row in enumerate(df["1500-5000_scaled"]):

    b = row/2
    t = 1/b
    for j in range(int(b)): # Repeat the chord for b times
        print(row,i,j, j*(1/b))
        for k in range(4): # For chord 
            print("->",time + i + j*(1/b) + (k*t/4))
            note = array_of_note_numbers[cur%len(array_of_note_numbers)]
            cur+=1
            
            MyMIDI.addNote(track, channel, note, time + i + j*(1/b) + (k*t/4), duration*t/4, volume)

with open("test.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)




2.0 5 0 0.0
-> 5.0
-> 5.25
-> 5.5
-> 5.75
2.0 6 0 0.0
-> 6.0
-> 6.25
-> 6.5
-> 6.75
2.0 19 0 0.0
-> 19.0
-> 19.25
-> 19.5
-> 19.75
2.0 20 0 0.0
-> 20.0
-> 20.25
-> 20.5
-> 20.75
2.0 26 0 0.0
-> 26.0
-> 26.25
-> 26.5
-> 26.75
2.0 27 0 0.0
-> 27.0
-> 27.25
-> 27.5
-> 27.75
2.0 33 0 0.0
-> 33.0
-> 33.25
-> 33.5
-> 33.75
2.0 36 0 0.0
-> 36.0
-> 36.25
-> 36.5
-> 36.75
2.0 37 0 0.0
-> 37.0
-> 37.25
-> 37.5
-> 37.75
2.0 40 0 0.0
-> 40.0
-> 40.25
-> 40.5
-> 40.75
2.0 44 0 0.0
-> 44.0
-> 44.25
-> 44.5
-> 44.75
2.0 45 0 0.0
-> 45.0
-> 45.25
-> 45.5
-> 45.75
2.0 50 0 0.0
-> 50.0
-> 50.25
-> 50.5
-> 50.75
2.0 53 0 0.0
-> 53.0
-> 53.25
-> 53.5
-> 53.75
2.0 54 0 0.0
-> 54.0
-> 54.25
-> 54.5
-> 54.75
2.0 56 0 0.0
-> 56.0
-> 56.25
-> 56.5
-> 56.75
2.0 57 0 0.0
-> 57.0
-> 57.25
-> 57.5
-> 57.75
2.0 58 0 0.0
-> 58.0
-> 58.25
-> 58.5
-> 58.75
2.0 60 0 0.0
-> 60.0
-> 60.25
-> 60.5
-> 60.75
2.0 61 0 0.0
-> 61.0
-> 61.25
-> 61.5
-> 61.75
2.0 64 0 0.0
-> 64.0
-> 64.25
-> 64.5
-> 64.75
2.0 65 0 0.0
-> 65.0
->